In [1]:
%matplotlib inline

import pandas as pd
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
from scipy.stats import pearsonr

from scipy.cluster.hierarchy import linkage
import scipy.spatial.distance as ssd

import seaborn as sns
import sys
from scipy.sparse import hstack, csr_matrix
from sklearn.feature_selection import mutual_info_classif
import os
from cnmf import cNMF

sys.path.append('../../Code/')
from utils import read_dataset_log


In [2]:
import pandas as pd
import scanpy as sc
import scipy.sparse as sp
from sklearn.decomposition import non_negative_factorization

class STARCAT(cNMF):
    def __init__(self, alpha=0.0, l1_ratio=0.0,  tpm_norm=True, var_norm=True, copy=True):
        """
        Class for running STARCAT on a query dataset. By default uses a standard reference
        database for human T-cells but you can optionally provide your own reference 
        object.
        """        
        self._nmf_kwargs = dict(
                        alpha_W=alpha,
                        alpha_H=0.0,
                        l1_ratio=l1_ratio,
                        beta_loss='frobenius',
                        solver='mu',
                        tol=1e-4,
                        max_iter=1000,
                        init='random',
                        update_H = False
                        )
        
        self.tpm_norm=tpm_norm
        self.var_norm=var_norm
        self.copy=copy
        
    def fit_transform(self, query, ref_spectra=None):
        """
        Takes an input data matrix and a fixed spectra and uses NNLS to find the optimal
        usage matrix. Generic kwargs for NMF are loaded from self.paths['nmf_run_parameters'].
        If input data are pandas.DataFrame, returns a DataFrame with row index matching X and
        columns index matching index of spectra

        Parameters
        ----------
        X : pandas.DataFrame or numpy.ndarray, cells X genes
            Non-negative expression data to fit spectra to

        spectra : pandas.DataFrame or numpy.ndarray, programs X genes
            Non-negative spectra of expression programs

        adata : query AnnData object
        ref_spectra : pd.DataFrame (#GEPs X #Genes) or None, optional (default=None, uses standard reference)
        """

        if ref_spectra is None:
            # Use default hard-coded reference (might live online or in the
            # TCAT code directory)
            print('Using default TCAT reference set')
            raise NotImplementedError("Not yet implemented.")
        else:
            self.ref = ref_spectra            
        self.ref = self.ref.astype(np.float32)
        
        num_nulls = self.ref.isnull().sum(axis=0)
        has_null = num_nulls>0
        nnull_genes = has_null.sum()
        if nnull_genes > 0:
            print('%d out of %d reference contain NaN in at least one GEP. Filtering them.' % (nnull_genes, self.ref.shape[1]))
            self.ref = self.ref.loc[:,~has_null]
            
        
        self.prepare_query(adata)
        rf_usages = self.fit_query_usage()
        return(rf_usages)
        
    
    def prepare_query(self, adata):
        """
        Load query dataset as AnnData object and optionally perform normalization.

        adata : query AnnData object
        tp10k_norm : boolean, optional (default=True, performs TP10K normalization)
        var_norm : boolean, optional (default=True, performs gene variance normalization)
        copy : boolean, optional (default=True, stores a copy of adata rather than modifying in place)

        """

        if self.copy:
            self.query = adata.copy()
        else:
            self.query = adata
            
        overlap_genes = list(set(self.ref.columns).intersection(set(adata.var.index)))
        print('%d out of %d genes in the reference overlap with the query' % (len(overlap_genes), self.ref.shape[1]))
        self.overlap_genes = overlap_genes
        self.query = self.query[:, self.overlap_genes]
            
        if self.tpm_norm:
            sc.pp.normalize_per_cell(self.query, counts_per_cell_after=1e6)
            
        if self.var_norm:
            sc.pp.scale(self.query, zero_center=False)
         
    def fit_query_usage(self):
        rf_usages = self.refit_usage(self.query.X, self.ref[self.overlap_genes].values,
                         self._nmf_kwargs.copy())          
        rf_usages = pd.DataFrame(rf_usages, index=self.query.obs.index,
                                 columns=self.ref.index)
        return(rf_usages)
        
    def refit_usage(self, X, spectra, nmf_kwargs):
        """
        Takes an input data matrix and a fixed spectra and uses NNLS to find the optimal
        usage matrix. Generic kwargs for NMF are loaded from self.paths['nmf_run_parameters'].
        If input data are pandas.DataFrame, returns a DataFrame with row index matching X and
        columns index matching index of spectra

        Parameters
        ----------
        X : pandas.DataFrame or numpy.ndarray, cells X genes
            Non-negative expression data to fit spectra to

        spectra : pandas.DataFrame or numpy.ndarray, programs X genes
            Non-negative spectra of expression programs
        """

        nmf_kwargs['H'] = spectra
        nmf_kwargs['n_components'] = spectra.shape[0]
        _, rf_usages = self._nmf(X, nmf_kwargs=nmf_kwargs)            
        return(rf_usages)

# Run TCAT on query dataset

In [3]:
toadd = '.20231016.FiltSingletons'

In [4]:
params = read_dataset_log('Dataset Paths')
params.index = params['dataset']
# params = params[params['dataset_type']=='discovery']
params

,dataset,usage_fn,gene_scores_fn,gene_tpm_fn,tcat_fn,tcat_fn_withsingletons,manual_gating_fn,metadata_fn,Processing notebook path,cNMF notebook path,...,cnmf_dir,k,dt,processed_forcnmf_fn,tpm_counts_for_cnmf_fn,raw_counts_filt_fn,raw_counts_fn,dataset_type,tissue_type,context_label
dataset,,,,,,,,,,,,,,,,,,,,,
AMP-RA,AMP-RA,/data/srlab1/TCAT/Data/PerDataset/AMPRA/AMPRA....,/data/srlab1/TCAT/Data/PerDataset/AMPRA/AMPRA....,/data/srlab1/TCAT/Data/PerDataset/AMPRA/AMPRA....,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,NaN,/data/srlab1/TCAT/Data/PerDataset/AMPRA/AMP_AD...,NaN,NaN,...,/data/srlab1/TCAT/Data/PerDataset/AMPRA,34.0,0_15,/data/srlab1/TCAT/Data/PerDataset/AMPRA/AMP_AD...,/data/srlab1/TCAT/Data/PerDataset/AMPRA/AMP_AD...,/data/srlab1/TCAT/Data/PerDataset/AMPRA/AMP_AD...,/data/srlab1/TCAT/Data/PerDataset/AMPRA/AMP_AD...,discovery,Synovium,RA+OA
Pan-Cancer,Pan-Cancer,/data/srlab1/TCAT/Data/PerDataset/Pancancer/Pa...,/data/srlab1/TCAT/Data/PerDataset/Pancancer/Pa...,/data/srlab1/TCAT/Data/PerDataset/Pancancer/Pa...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,NaN,/data/srlab1/TCAT/Data/PerDataset/Pancancer/pa...,NaN,NaN,...,/data/srlab1/TCAT/Data/PerDataset/Pancancer,38.0,0.15,/data/srlab1/TCAT/Data/PerDataset/Pancancer/pa...,NaN,/data/srlab1/TCAT/Data/PerDataset/Pancancer/pa...,/data/srlab1/TCAT/Data/PerDataset/Pancancer/pa...,discovery,Pan-Tissue,Cancer+Healthy
TBRU,TBRU,/data/srlab1/TCAT/Data/PerDataset/TBRU/TBRU.20...,/data/srlab1/TCAT/Data/PerDataset/TBRU/TBRU.20...,/data/srlab1/TCAT/Data/PerDataset/TBRU/TBRU.20...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,NaN,/data/srlab1/TCAT/Data/PerDataset/TBRU/tbru_ex...,NaN,NaN,...,/data/srlab1/TCAT/Data/PerDataset/TBRU,36.0,0_20,/data/srlab1/TCAT/Data/PerDataset/TBRU/tbru_ex...,/data/srlab1/TCAT/Data/PerDataset/TBRU/tbru_ex...,/data/srlab1/TCAT/Data/PerDataset/TBRU/tbru_ex...,/data/srlab1/TCAT/Data/PerDataset/TBRU/tbru_ex...,discovery,Blood,Healthy
HIV-Vaccine,HIV-Vaccine,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/HIVV...,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/HIVV...,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/HIVV...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/Manu...,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/haoe...,/data/srlab1/TCAT/Analysis/PerDataset/HaoEtAl/...,/data/srlab1/TCAT/Analysis/PerDataset/HaoEtAl/...,...,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl,31.0,0_15,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/haoe...,NaN,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/haoe...,/data/srlab1/TCAT/Data/PerDataset/HaoEtAl/haoe...,discovery,Blood,Post-Vaccine+Healthy
UK-Covid,UK-Covid,/data/srlab1/TCAT/Data/PerDataset/UKCOVID/UKCO...,/data/srlab1/TCAT/Data/PerDataset/UKCOVID/UKCO...,/data/srlab1/TCAT/Data/PerDataset/UKCOVID/UKCO...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,NaN,/data/srlab1/TCAT/Data/PerDataset/UKCOVID/hani...,NaN,NaN,...,/data/srlab1/TCAT/Data/PerDataset/UKCOVID,44.0,0_20,/data/srlab1/TCAT/Data/PerDataset/UKCOVID/hani...,NaN,/data/srlab1/TCAT/Data/PerDataset/UKCOVID/hani...,/data/srlab1/TCAT/Data/PerDataset/UKCOVID/hani...,discovery,Blood,Covid-19+Healthy
COMBAT,COMBAT,/data/srlab1/TCAT/Data/PerDataset/COMBAT/COMBA...,/data/srlab1/TCAT/Data/PerDataset/COMBAT/COMBA...,/data/srlab1/TCAT/Data/PerDataset/COMBAT/COMBA...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,/data/srlab1/TCAT/Data/TCAT/TCAT_Usage.2023101...,/data/srlab1/TCAT/Data/PerDataset/COMBAT/Manua...,/data/srlab1/TCAT/Data/PerDataset/COMBAT/COMBA...,NaN,NaN,...,/data/srlab1/TCAT/Data/PerDataset/COMBAT,35.0,0_15,/data/srlab1/TCAT/Data/PerDataset/COMBAT/COMBA...,NaN,/data/srlab1/TCAT/Data/PerDataset/COMBAT/COMBA...,/data/srlab1/TCAT/Data/PerDataset/COMBAT/COMBA...,discovery,Blood,Covid-19+He

In [5]:
dataset = 'Sparks'
adata = sc.read(params.loc[dataset, 'raw_counts_fn'])

In [6]:
metagep_params = read_dataset_log('cGEP Paths').iloc[0,:]
metagep_params

clustering_fn                                /data/srlab1/TCAT/Data/TCAT/cluster_groups.pai...
clustering_with_singletons_fn                /data/srlab1/TCAT/Data/TCAT/cluster_groups.pai...
merged_hvgs                                  /data/srlab1/TCAT/Data/TCAT/MergedHVG_UnionAll...
tpm_varnorm_spectra                          /data/srlab1/TCAT/Data/TCAT/merged_spectra.Gen...
tpm_varnorm_spectra_withsingletons           /data/srlab1/TCAT/Data/TCAT/merged_spectra.Gen...
tpm_renorm_varnorm_spectra                   /data/srlab1/TCAT/Data/TCAT/merged_spectra.Gen...
tpm_renorm_varnorm_spectra_withsingletons    /data/srlab1/TCAT/Data/TCAT/merged_spectra.Gen...
scores_spectra                               /data/srlab1/TCAT/Data/TCAT/merged_spectra.Gen...
scores_spectra_withsingletons                /data/srlab1/TCAT/Data/TCAT/merged_spectra.Gen...
correlation_matrix_tpm_renorm_varnorm        /data/srlab1/TCAT/Data/TCAT/R.TPMrenorm.VarNor...
correlation_matrix_spectra                   /data

In [5]:
metagep_params['clustering_fn']

'/data/srlab1/TCAT/Data/TCAT/cluster_groups.pairwiseCorr.TPMrenorm.VarNorm.HVGunion.FiltSingletons.20231016.txt '

In [6]:
metagep_params['tpm_renorm_varnorm_spectra']

'/data/srlab1/TCAT/Data/TCAT/merged_spectra.Gene_TPM_Renorm_VarNorm.Gene_Intersection.TPMrenorm.VarNorm.HVGunion.FiltSingletons.Mean.20231016.tsv'

In [7]:
with_singletons = False

In [8]:
if with_singletons:
    ref_fn = metagep_params['tpm_renorm_varnorm_spectra_withsingletons']
else:
    ref_fn = metagep_params['tpm_renorm_varnorm_spectra']
hvg_fn = metagep_params['merged_hvgs']

ref = pd.read_csv(ref_fn, sep='\t', index_col=0)
hvgs = pd.read_csv(hvg_fn, sep='\t', header=None)[0]
ref = ref[hvgs]
ref.iloc[:5,:5]

,A1BG,AARD,AARSD1,ABCA1,ABCB1
CellCycle-G2M,2.032614,22.965553,17.423538,3.478179,2.297279
CD4-Naive_Translation,35.445284,0.000000,9.245894,0.477994,0.000000
HLA,18.192998,14.632671,2.686475,3.937182,0.000000
ISG,0.436212,0.000000,18.078198,17.354505,0.000000
MALAT1_Mito,10.293049,0.000000,52.669894,14.615502,3.341488


In [17]:
ref_fn

'/data/srlab1/TCAT/Data/TCAT/merged_spectra.Gene_TPM_Renorm_VarNorm.Gene_Intersection.TPMrenorm.VarNorm.HVGunion.FiltSingletons.Mean.20231016.tsv'

In [12]:
usage_all = {}
for dataset in ['Sparks']:
    # adata = sc.read(params.loc[dataset, 'raw_counts_fn'])
    tmod = STARCAT(alpha=0, l1_ratio=0, tpm_norm=False, var_norm=True, copy=True)
    usage_all[dataset] = tmod.fit_transform(adata, ref_spectra=ref)

3412 out of 3412 genes in the reference overlap with the query


/PHShome/mc1070/anaconda3/envs/cnmf_env2v2/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:843: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [13]:
toadd

'.20231016.FiltSingletons'

In [14]:
! pwd

/data/srlab1/TCAT/Analysis/TCAT


In [16]:
outfnbase = '../../Data/TCAT/TCAT_Usage%s.{d}.tsv' % toadd
for dataset in ['Sparks']:
    outfn = outfnbase.format(d=dataset)
    print(outfn)
    usage_all[dataset].to_csv(outfn, sep='\t')


../../Data/TCAT/TCAT_Usage.20231016.FiltSingletons.Sparks.tsv


In [ ]:
! ls /data/srlab1/TCAT/Data/TCAT/TCAT_Usage*

In [11]:
# Rescale usage matrix
scaler = 1e2
usage_all_scaled = {}
for dataset in ['Sparks']:
    print(np.sum(usage_all[dataset], axis = 1).mean())
    outfn = outfnbase.format(d=dataset)
    outfn = outfn.replace('%s.tsv' % dataset, 'Rescaled.%s.tsv' % dataset)
    print(outfn)
    usage_all_scaled[dataset] = usage_all[dataset]*scaler
    usage_all_scaled[dataset].to_csv(outfn, sep='\t')


0.008518775348079545
../../Data/TCAT/TCAT_Usage.20231016.FiltSingletons.Rescaled.Sparks.tsv
